In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer
import seaborn as sns

In [2]:
#import and show the first 5 rows of the data
raw_data = pd.read_csv('https://raw.githubusercontent.com/askidelsky/DAV-6150/main/Homework/Project%201/Project1_Data.csv')
raw_data.head()

,report_school_year,aggregation_index,aggregation_type,aggregation_name,nrc_code,nrc_desc,county_code,county_name,nyc_ind,membership_desc,...,reg_adv_cnt,reg_adv_pct,non_diploma_credential_cnt,non_diploma_credential_pct,still_enr_cnt,still_enr_pct,ged_cnt,ged_pct,dropout_cnt,dropout_pct
0,2018-19,3,District,ALBANY CITY SCHOOL DISTRICT,3,Urban-Suburban High Needs,1,ALBANY,0,2013 Total Cohort - 6 Year Outcome,...,91,14%,16,2%,30,5%,0,0%,148,22%
1,2018-19,3,District,ALBANY CITY SCHOOL DISTRICT,3,Urban-Suburban High Needs,1,ALBANY,0,2013 Total Cohort - 6 Year Outcome,...,47,15%,2,1%,11,3%,0,0%,65,20%
2,2018-19,3,District,ALBANY CITY SCHOOL DISTRICT,3,Urban-Suburban High Needs,1,ALBANY,0,2013 Total Cohort - 6 Year Outcome,...,44,13%,14,4%,19,6%,0,0%,83,25%
3,2018-19,3,District,ALBANY CITY SCHOOL DISTRICT,3,Urban-Suburban High Needs,1,ALBANY,0,2013 Total Cohort - 6 Year Outcome,...,-,-,-,-,-,-,-,-,-,-
4,2018-19,3,District,ALBANY CITY SCHOOL DISTRICT,3,Urban-Suburban High Needs,1,ALBANY,0,2013 Total Cohort - 6 Year Outcome,...,23,6%,10,3%,18,5%,0,0%,91,25%


In [3]:
#data consists of 73,152 rows and 29 columns, each column representing statistics for a different csubsection of students
raw_data.shape

(73152, 29)

In [4]:
#note that any field enging in "pct" is simply the corresponding cnt divided by enroll_cnt. 
#having this equation is useful since all numerical values are stored in string form and those fields will have to be recalculated
#This is also useful to know as it will be applied to field selection later. 
#I will likely use pcnt as a variable rather than cnt since in most cases it is more normalized
raw_data.iloc[0, 12:]

enroll_cnt                    658
grad_cnt                      464
grad_pct                      71%
local_cnt                      63
local_pct                     10%
reg_cnt                       310
reg_pct                       47%
reg_adv_cnt                    91
reg_adv_pct                   14%
non_diploma_credential_cnt     16
non_diploma_credential_pct     2%
still_enr_cnt                  30
still_enr_pct                  5%
ged_cnt                         0
ged_pct                        0%
dropout_cnt                   148
dropout_pct                   22%
Name: 0, dtype: object

In [5]:
lst = 0
for i in range(len(raw_data)):
    if raw_data.enroll_cnt[i].isnumeric()==False and raw_data.grad_cnt[i].isnumeric()==False and raw_data.local_cnt[i].isnumeric()==False and raw_data.reg_cnt[i].isnumeric()==False and raw_data.reg_adv_cnt[i].isnumeric()==False and raw_data.non_diploma_credential_cnt[i].isnumeric()==False and raw_data.still_enr_cnt[i].isnumeric()==False and raw_data.ged_cnt[i].isnumeric()==False and raw_data.dropout_cnt[i].isnumeric()==False:
        lst += 1
lst

33478

In [6]:
types = {}
for i in range(len(raw_data.columns)):
    types[raw_data.columns[i]] = type(raw_data.iloc[0,i])
types

{'report_school_year': str,
 'aggregation_index': numpy.int64,
 'aggregation_type': str,
 'aggregation_name': str,
 'nrc_code': numpy.int64,
 'nrc_desc': str,
 'county_code': numpy.int64,
 'county_name': str,
 'nyc_ind': numpy.int64,
 'membership_desc': str,
 'subgroup_code': numpy.int64,
 'subgroup_name': str,
 'enroll_cnt': str,
 'grad_cnt': str,
 'grad_pct': str,
 'local_cnt': str,
 'local_pct': str,
 'reg_cnt': str,
 'reg_pct': str,
 'reg_adv_cnt': str,
 'reg_adv_pct': str,
 'non_diploma_credential_cnt': str,
 'non_diploma_credential_pct': str,
 'still_enr_cnt': str,
 'still_enr_pct': str,
 'ged_cnt': str,
 'ged_pct': str,
 'dropout_cnt': str,
 'dropout_pct': str}

In [7]:
cats = raw_data[['report_school_year','aggregation_index','aggregation_type','aggregation_name','nrc_code','nrc_desc','county_code',
         'county_name','nyc_ind','membership_desc','subgroup_code','subgroup_name']]

In [8]:
#looking at unique values, can see that report_school_year, aggregation_index, and aggregation_type can all be taken out since 
#they are all one uniform value and therefore will not contribute in any meaningful way to our analysis
vals = {}
for i in range(len(cats.columns)):
    vals[cats.columns[i]] = cats.iloc[:,i].unique()
vals

{'report_school_year': array(['2018-19'], dtype=object),
 'aggregation_index': array([3], dtype=int64),
 'aggregation_type': array(['District'], dtype=object),
 'aggregation_name': array(['ALBANY CITY SCHOOL DISTRICT',
        'BERNE-KNOX-WESTERLO CENTRAL SCHOOL DISTRICT',
        'BETHLEHEM CENTRAL SCHOOL DISTRICT',
        'RAVENA-COEYMANS-SELKIRK CENTRAL SCHOOL DISTRICT',
        'COHOES CITY SCHOOL DISTRICT',
        'SOUTH COLONIE CENTRAL SCHOOL DISTRICT', 'NORTH COLONIE CSD',
        'GREEN ISLAND UNION FREE SCHOOL DISTRICT',
        'GUILDERLAND CENTRAL SCHOOL DISTRICT',
        'VOORHEESVILLE CENTRAL SCHOOL DISTRICT',
        'WATERVLIET CITY SCHOOL DISTRICT',
        'ALFRED-ALMOND CENTRAL SCHOOL DISTRICT',
        'ANDOVER CENTRAL SCHOOL DISTRICT',
        'GENESEE VALLEY CENTRAL SCHOOL DISTRICT',
        'BELFAST CENTRAL SCHOOL DISTRICT',
        'CANASERAGA CENTRAL SCHOOL DISTRICT',
        'FRIENDSHIP CENTRAL SCHOOL DISTRICT',
        'FILLMORE CENTRAL SCHOOL DISTRICT',
  

In [9]:
cats.drop(columns=['report_school_year','aggregation_index','aggregation_type'], inplace=True)
raw_data.drop(columns=['report_school_year','aggregation_index','aggregation_type'], inplace=True)

C:\Users\askidelsky\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [17]:
raw_data.replace('-', np.NaN, inplace=True)
raw_data[raw_data.enroll_cnt.isna()==True].head()

,aggregation_name,nrc_code,nrc_desc,county_code,county_name,nyc_ind,membership_desc,subgroup_code,subgroup_name,enroll_cnt,...,reg_adv_cnt,reg_adv_pct,non_diploma_credential_cnt,non_diploma_credential_pct,still_enr_cnt,still_enr_pct,ged_cnt,ged_pct,dropout_cnt,dropout_pct
3,ALBANY CITY SCHOOL DISTRICT,3,Urban-Suburban High Needs,1,ALBANY,0,2013 Total Cohort - 6 Year Outcome,4,American Indian/Alaska Native,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ALBANY CITY SCHOOL DISTRICT,3,Urban-Suburban High Needs,1,ALBANY,0,2013 Total Cohort - 6 Year Outcome,9,Multiracial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,ALBANY CITY SCHOOL DISTRICT,3,Urban-Suburban High Needs,1,ALBANY,0,2013 Total Cohort - 6 Year Outcome,14,Formerly English Language Learner,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,ALBANY CITY SCHOOL DISTRICT,3,Urban-Suburban High Needs,1,ALBANY,0,2013 Total Cohort - 6 Year Outcome,17,Migrant,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,ALBANY CITY SCHOOL DISTRICT,3,Urban-Suburban High Needs,1,ALBANY,0,2013 Total Cohort - 6 Year Outcome,18,Not Migrant,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


https://vitalflux.com/pandas-impute-missing-values-mean-median-mode/

In [19]:
wNas=raw_data[['enroll_cnt','grad_cnt','local_cnt','reg_cnt','reg_adv_cnt','non_diploma_credential_cnt','still_enr_cnt',
               'ged_cnt','dropout_cnt']]

In [24]:
nas = raw_data[raw_data.enroll_cnt.isna()==True]

In [28]:
nas.corr()

,nrc_code,county_code,nyc_ind,subgroup_code
nrc_code,1.000000,0.176712,-0.558326,-0.046868
county_code,0.176712,1.000000,-0.019516,0.016165
nyc_ind,-0.558326,-0.019516,1.000000,0.096927
subgroup_code,-0.046868,0.016165,0.096927,1.000000


In [30]:
nas.drop(columns=['nyc_ind'], inplace=True)

C:\Users\askidelsky\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
